# Sentinel-1 GRD Data with EOPFZARR Driver

This notebook demonstrates accessing Sentinel-1 Ground Range Detected (GRD) SAR data using the EOPFZARR GDAL driver.

## Overview

- **Product Type**: S01SEWGRD (Sentinel-1C Extra Wide mode Ground Range Detected)
- **Dataset**: S1C_EW_GRDM_1SSH from EODC
- **Key Features**: SAR amplitude data with Ground Control Points (GCPs)

## Sentinel-1 GRD Structure

Unlike optical sensors (Sentinel-2/3), Sentinel-1 uses:
- **Sparse GCP grid** instead of dense lat/lon arrays
- **Ground Control Points** at `conditions/gcp/latitude` and `conditions/gcp/longitude`
- **Measurement data** at `measurements/grd` (SAR backscatter amplitude)

## 1. Setup and Imports

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
import json
from urllib.request import urlopen

# Enable GDAL exceptions
gdal.UseExceptions()

print("✅ Imports successful!")
print(f"GDAL version: {gdal.__version__}")

## 2. Dataset Configuration

We'll use a Sentinel-1C Extra Wide mode GRD product from EODC.

In [ ]:
# Sentinel-1 GRD dataset URL
base_url = (
    "https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202601-s01sewgrm-global/"
    "29/products/cpm_v262/S1C_EW_GRDM_1SSH_20260129T124421_20260129T124526_"
    "006118_00C473_F440.zarr"
)

# Construct EOPFZARR path
zarr_path = f'EOPFZARR:"/vsicurl/{base_url}"'

print("Dataset Information:")
print("=" * 80)
print(f"Platform: Sentinel-1C")
print(f"Product Type: S1C_EW_GRDM_1SSH (Extra Wide Ground Range Medium Resolution)")
print(f"Acquisition: 2026-01-29 12:44:21 to 12:45:26 UTC")
print(f"Polarization: HH")
print(f"\nZarr path: {zarr_path}")
print("=" * 80)

## 3. Open Root Dataset and List Subdatasets

First, let's open the root dataset to discover all available subdatasets.

In [ ]:
# Open root dataset
print("Opening root dataset...\n")
root_ds = gdal.Open(zarr_path)

if root_ds is None:
    print("❌ Failed to open dataset!")
else:
    print("✅ Root dataset opened successfully\n")
    
    # Get subdatasets
    subdatasets = root_ds.GetMetadata("SUBDATASETS")
    
    # Parse subdataset list
    sub_list = []
    i = 1
    while f"SUBDATASET_{i}_NAME" in subdatasets:
        name = subdatasets[f"SUBDATASET_{i}_NAME"]
        desc = subdatasets.get(f"SUBDATASET_{i}_DESC", "")
        sub_list.append((name, desc))
        i += 1
    
    print(f"Total subdatasets: {len(sub_list)}\n")
    
    # Categorize subdatasets
    measurements = []
    gcp_arrays = []
    other = []
    
    for name, desc in sub_list:
        # Extract path from EOPFZARR format
        if '":/' in name:
            path = name.split('":/')[1]
        else:
            path = name
        
        if 'measurements' in path:
            measurements.append((path, name))
        elif 'conditions/gcp' in path:
            gcp_arrays.append((path, name))
        else:
            other.append((path, name))
    
    print("📊 Measurement Arrays:")
    print("=" * 80)
    for path, _ in measurements:
        print(f"  {path}")
    
    print("\n🌍 Ground Control Point (GCP) Arrays:")
    print("=" * 80)
    for path, _ in gcp_arrays:
        print(f"  {path}")
    
    print(f"\n📋 Other Arrays: {len(other)}")
    if len(other) <= 10:
        for path, _ in other:
            print(f"  {path}")
    else:
        for path, _ in other[:10]:
            print(f"  {path}")
        print(f"  ... and {len(other) - 10} more")

## 4. Inspect Root Dataset Metadata

Check what geospatial metadata is available at the root level.

In [ ]:
print("Root Dataset Metadata:")
print("=" * 80)
print(f"Driver: {root_ds.GetDriver().ShortName}")
print(f"Size: {root_ds.RasterXSize} x {root_ds.RasterYSize}")
print(f"Bands: {root_ds.RasterCount}")

# Check for CRS
srs = root_ds.GetProjection()
if srs:
    print(f"\nProjection: {srs[:100]}...")
else:
    print("\nProjection: None")

# Check for GeoTransform
gt = root_ds.GetGeoTransform()
if gt and gt != (0, 1, 0, 0, 0, 1):
    print(f"\nGeoTransform: {gt}")
    print(f"  Origin: ({gt[0]:.4f}, {gt[3]:.4f})")
    print(f"  Pixel Size: ({gt[1]:.6f}, {gt[5]:.6f})")
else:
    print("\nGeoTransform: Default (no georeferencing)")

# Check for metadata domains
domains = root_ds.GetMetadataDomainList()
print(f"\nMetadata Domains: {domains}")

# Check for GEOLOCATION metadata
if 'GEOLOCATION' in (domains or []):
    geoloc_md = root_ds.GetMetadata('GEOLOCATION')
    print("\n🌍 GEOLOCATION Metadata:")
    for key, value in geoloc_md.items():
        print(f"  {key}: {value}")
else:
    print("\n⚠️  No GEOLOCATION metadata found at root level")

print("=" * 80)

## 5. Open GRD Measurement Data

Now let's open the main SAR amplitude data (GRD = Ground Range Detected).

In [ ]:
# Find the GRD measurement array
grd_path = None
for path, full_name in measurements:
    if path.endswith('/grd'):
        grd_path = full_name
        break

if grd_path is None:
    print("❌ GRD measurement array not found!")
else:
    print(f"Opening: {grd_path.split('":/')[1]}\n")
    
    grd_ds = gdal.Open(grd_path)
    
    if grd_ds is None:
        print("❌ Failed to open GRD dataset!")
    else:
        print("✅ GRD dataset opened successfully\n")
        
        print("GRD Measurement Metadata:")
        print("=" * 80)
        print(f"Driver: {grd_ds.GetDriver().ShortName}")
        print(f"Dimensions: {grd_ds.RasterXSize} x {grd_ds.RasterYSize} pixels")
        print(f"Bands: {grd_ds.RasterCount}")
        
        # Band information
        band = grd_ds.GetRasterBand(1)
        print(f"\nBand 1:")
        print(f"  Data Type: {gdal.GetDataTypeName(band.DataType)}")
        print(f"  Block Size: {band.GetBlockSize()}")
        print(f"  NoData Value: {band.GetNoDataValue()}")
        
        # Check for CRS
        srs = grd_ds.GetProjection()
        if srs:
            print(f"\nProjection: {srs[:80]}...")
        else:
            print("\nProjection: None")
        
        # Check for GeoTransform
        gt = grd_ds.GetGeoTransform()
        if gt and gt != (0, 1, 0, 0, 0, 1):
            print(f"\nGeoTransform: {gt}")
            print(f"  Origin: ({gt[0]:.4f}, {gt[3]:.4f})")
            print(f"  Pixel Size: ({gt[1]:.6f}, {gt[5]:.6f})")
        else:
            print("\nGeoTransform: Default (no georeferencing)")
        
        # Check for GEOLOCATION metadata
        domains = grd_ds.GetMetadataDomainList()
        if 'GEOLOCATION' in (domains or []):
            geoloc_md = grd_ds.GetMetadata('GEOLOCATION')
            print("\n🌍 GEOLOCATION Metadata:")
            for key, value in geoloc_md.items():
                print(f"  {key}: {value}")
        else:
            print("\n⚠️  No GEOLOCATION metadata found")
        
        # Check for GCPs
        gcp_count = grd_ds.GetGCPCount()
        if gcp_count > 0:
            print(f"\n🎯 Ground Control Points: {gcp_count}")
            gcps = grd_ds.GetGCPs()
            print(f"\nFirst 5 GCPs:")
            for i, gcp in enumerate(gcps[:5]):
                print(f"  GCP {i+1}: Pixel({gcp.GCPPixel:.1f}, {gcp.GCPLine:.1f}) -> "
                      f"Geo({gcp.GCPX:.4f}, {gcp.GCPY:.4f})")
        else:
            print(f"\n⚠️  No GCPs found (count: {gcp_count})")
        
        print("=" * 80)

## 6. Inspect GCP Arrays Directly

Let's open and examine the GCP latitude/longitude arrays.

In [ ]:
# Find GCP arrays
gcp_lat_path = None
gcp_lon_path = None

for path, full_name in gcp_arrays:
    if 'latitude' in path:
        gcp_lat_path = full_name
    elif 'longitude' in path:
        gcp_lon_path = full_name

if gcp_lat_path and gcp_lon_path:
    print("Opening GCP arrays...\n")
    
    # Open latitude
    lat_ds = gdal.Open(gcp_lat_path)
    print(f"✅ Latitude: {gcp_lat_path.split('":/')[1]}")
    print(f"   Dimensions: {lat_ds.RasterXSize} x {lat_ds.RasterYSize}")
    lat_array = lat_ds.ReadAsArray()
    print(f"   Shape: {lat_array.shape}")
    print(f"   Range: [{lat_array.min():.4f}, {lat_array.max():.4f}]°")
    
    # Open longitude
    lon_ds = gdal.Open(gcp_lon_path)
    print(f"\n✅ Longitude: {gcp_lon_path.split('":/')[1]}")
    print(f"   Dimensions: {lon_ds.RasterXSize} x {lon_ds.RasterYSize}")
    lon_array = lon_ds.ReadAsArray()
    print(f"   Shape: {lon_array.shape}")
    print(f"   Range: [{lon_array.min():.4f}, {lon_array.max():.4f}]°")
    
    print("\n" + "=" * 80)
    print("GCP Grid Structure:")
    print("=" * 80)
    print(f"GRD Data: {grd_ds.RasterXSize} x {grd_ds.RasterYSize} pixels (full resolution)")
    print(f"GCP Grid: {lat_array.shape[1]} x {lat_array.shape[0]} points (sparse grid)")
    print(f"\nSampling Ratio:")
    print(f"  X: 1 GCP every ~{grd_ds.RasterXSize / lat_array.shape[1]:.0f} pixels")
    print(f"  Y: 1 GCP every ~{grd_ds.RasterYSize / lat_array.shape[0]:.0f} pixels")
    print("\n⚠️  This is a SPARSE geolocation grid, not dense per-pixel coordinates")
    print("=" * 80)
else:
    print("❌ GCP latitude/longitude arrays not found!")

## 7. Visualize GCP Grid

Let's visualize the sparse GCP grid to understand its distribution.

In [ ]:
if gcp_lat_path and gcp_lon_path:
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # Plot 1: Latitude grid
    im1 = axes[0].imshow(lat_array, cmap='coolwarm', aspect='auto')
    axes[0].set_title('GCP Latitude Grid\n(Sparse Control Points)', fontweight='bold')
    axes[0].set_xlabel('GCP X Index')
    axes[0].set_ylabel('GCP Y Index')
    plt.colorbar(im1, ax=axes[0], label='Latitude (°)')
    
    # Plot 2: Longitude grid
    im2 = axes[1].imshow(lon_array, cmap='coolwarm', aspect='auto')
    axes[1].set_title('GCP Longitude Grid\n(Sparse Control Points)', fontweight='bold')
    axes[1].set_xlabel('GCP X Index')
    axes[1].set_ylabel('GCP Y Index')
    plt.colorbar(im2, ax=axes[1], label='Longitude (°)')
    
    # Plot 3: Geographic distribution
    axes[2].scatter(lon_array.flatten(), lat_array.flatten(), 
                   c=np.arange(lat_array.size), cmap='viridis', s=30, alpha=0.6)
    axes[2].set_title('Geographic Distribution of GCPs', fontweight='bold')
    axes[2].set_xlabel('Longitude (°)')
    axes[2].set_ylabel('Latitude (°)')
    axes[2].grid(True, alpha=0.3)
    axes[2].set_aspect('equal', adjustable='box')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ GCP grid visualization complete")
else:
    print("⚠️  Cannot visualize - GCP arrays not loaded")

## 8. Read and Visualize SAR Data

Let's read a subset of the GRD data and visualize the SAR backscatter.

In [ ]:
if grd_ds:
    print("Reading SAR data subset...\n")
    
    # Read a subset for visualization (center region)
    x_offset = grd_ds.RasterXSize // 4
    y_offset = grd_ds.RasterYSize // 4
    x_size = grd_ds.RasterXSize // 2
    y_size = grd_ds.RasterYSize // 2
    
    print(f"Reading subset:")
    print(f"  Offset: ({x_offset}, {y_offset})")
    print(f"  Size: {x_size} x {y_size} pixels")
    
    # Read data
    sar_data = grd_ds.GetRasterBand(1).ReadAsArray(
        xoff=x_offset, yoff=y_offset,
        win_xsize=x_size, win_ysize=y_size
    )
    
    print(f"\n✅ Data read successfully")
    print(f"   Shape: {sar_data.shape}")
    print(f"   Data type: {sar_data.dtype}")
    print(f"   Value range: [{sar_data.min()}, {sar_data.max()}]")
    print(f"   Memory: {sar_data.nbytes / 1024 / 1024:.2f} MB")
    
    # Convert to float and apply log scale for visualization
    sar_data_float = sar_data.astype(np.float32)
    sar_data_float[sar_data_float == 0] = np.nan  # Mask zeros
    sar_data_db = 10 * np.log10(sar_data_float + 1)  # Log scale (dB-like)
    
    print(f"\nStatistics (log scale):")
    print(f"   Min: {np.nanmin(sar_data_db):.2f}")
    print(f"   Max: {np.nanmax(sar_data_db):.2f}")
    print(f"   Mean: {np.nanmean(sar_data_db):.2f}")
    print(f"   Std: {np.nanstd(sar_data_db):.2f}")
else:
    print("⚠️  GRD dataset not available")

## 9. Visualize SAR Backscatter

In [ ]:
if 'sar_data_db' in locals():
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    # Plot 1: SAR backscatter
    im1 = axes[0].imshow(sar_data_db, cmap='gray', vmin=np.nanpercentile(sar_data_db, 2),
                         vmax=np.nanpercentile(sar_data_db, 98))
    axes[0].set_title('Sentinel-1 GRD Backscatter (HH Polarization)\nLog Scale', 
                      fontsize=13, fontweight='bold')
    axes[0].set_xlabel('Range (pixels)')
    axes[0].set_ylabel('Azimuth (pixels)')
    cbar1 = plt.colorbar(im1, ax=axes[0], label='Backscatter (dB-like)')
    
    # Plot 2: Histogram
    valid_data = sar_data_db[~np.isnan(sar_data_db)].flatten()
    axes[1].hist(valid_data, bins=100, edgecolor='black', alpha=0.7, color='steelblue')
    axes[1].set_title('Backscatter Distribution', fontsize=13, fontweight='bold')
    axes[1].set_xlabel('Backscatter (dB-like)')
    axes[1].set_ylabel('Frequency')
    axes[1].grid(True, alpha=0.3)
    axes[1].axvline(np.nanmean(sar_data_db), color='red', linestyle='--', 
                   linewidth=2, label=f'Mean: {np.nanmean(sar_data_db):.2f}')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    print("✅ SAR visualization complete")
    print("\nNote: Bright areas = strong backscatter (urban, rough surfaces, ships)")
    print("      Dark areas = weak backscatter (calm water, smooth surfaces)")
else:
    print("⚠️  SAR data not available for visualization")

## 10. Summary and Findings

### Current Status of Sentinel-1 Support

#### ✅ What Works:
1. **Dataset Discovery**: Root dataset opens and lists all subdatasets
2. **SAR Data Access**: GRD measurement data can be read and visualized
3. **GCP Arrays**: Latitude/longitude GCP grids are accessible as subdatasets
4. **Basic Metadata**: Product information and array dimensions are available

#### ⚠️ Current Limitations:
1. **No Automatic GCP Integration**: The driver doesn't automatically attach GCPs from `conditions/gcp/` arrays to the measurement dataset
2. **No GEOLOCATION Metadata**: The sparse GCP grid is not exposed via GDAL's GEOLOCATION metadata domain
3. **Limited Georeferencing**: Without GCPs, precise georeferencing requires manual interpolation

### Sentinel-1 vs Sentinel-3 Geolocation

| Aspect | Sentinel-3 OLCI | Sentinel-1 GRD |
|--------|----------------|----------------|
| **Geolocation Type** | Dense lat/lon arrays | Sparse GCP grid |
| **Array Size** | Same as measurement data | Much smaller (23x21 vs 10923x10457) |
| **Coverage** | Every pixel | ~1 point per 500 pixels |
| **Driver Support** | ✅ Fully automatic | ⚠️ Manual access only |
| **GEOLOCATION Metadata** | ✅ Yes | ❌ No |

### Next Steps for Full Support:

To enable automatic georeferencing for Sentinel-1, the driver would need to:
1. Detect GCP arrays in `conditions/gcp/` group
2. Read the sparse lat/lon grids
3. Attach them as GDAL GCPs to the measurement dataset
4. Enable reprojection and coordinate transformations

### Workarounds:

For now, users can:
1. Manually read GCP arrays as shown in this notebook
2. Interpolate the sparse grid to full resolution if needed
3. Use external tools (e.g., SNAP) for full geometric correction

## 11. Cleanup

In [ ]:
# Close datasets
if 'root_ds' in locals():
    root_ds = None
if 'grd_ds' in locals():
    grd_ds = None
if 'lat_ds' in locals():
    lat_ds = None
if 'lon_ds' in locals():
    lon_ds = None

print("✅ All datasets closed")